In [10]:
%%bash
set -euo pipefail

# Make the 'module' command available (Lmod or Environment Modules)
if [ -f /etc/profile.d/lmod.sh ]; then source /etc/profile.d/lmod.sh; fi
if [ -f /etc/profile.d/modules.sh ]; then source /etc/profile.d/modules.sh; fi
# Fallback some clusters use:
if [ -f /usr/share/Modules/init/bash ]; then source /usr/share/Modules/init/bash; fi

module load matlab          # or: module load matlab/R2024a
which matlab                # sanity check

# cd to your working dir if needed:
# cd /global/u2/d/devard/dl

matlab -nodisplay -nosplash -nodesktop -batch \
"try, elevation=0; southern=false; surf='surf_mete.mat'; prof='morning_prof.mat'; process_input(elevation, southern, surf, prof); catch e, disp(getReport(e)); exit(1); end; exit"


/global/common/software/nersc9/matlab/R2023b/bin/matlab


In [4]:
import os
import numpy as np
import scipy.io
from scipy.io import loadmat
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import pickle
import scipy.io as sio
from netCDF4 import Dataset

rescale = 0  # set to 1 to refit the scaler if current meteorology ranges differ significantly from the training data

def build_cldtrain(y_train_pred, y_train_pred1, y_train_pred2, pp, cldvel):
    """
    Convert model predictions into PBL & cloud profile arrays.

    Parameters
    ----------
    y_train_pred : array_like, shape (LM*13, F) with F >= 12
        Feature predictions (11 CF features in cols 0..10; thickness in col 11).
    y_train_pred1 : array_like, shape (LM*13,) or (1, LM*13)
        BLC flags (linearized).
    y_train_pred2 : array_like, shape (LM*13,) or (1, LM*13)
        Cloud-base heights (linearized).
    pp : array_like, shape (LM*13,) or (1, LM*13)
        Baseline PBLH (pp2), linearized.
    cldvel : array_like, shape (V,)
        Cloud levels.

    Returns
    -------
    pbltrain2 : ndarray, shape (LM, 24)
    blc_f     : ndarray, shape (LM, 24)
    cldcbh1   : ndarray, shape (LM, 24)
    cldcf     : ndarray, shape (LM, 24, 11)
    cldth     : ndarray, shape (LM, 24)
    cldtrain  : ndarray, shape (LM, 24, V)
    """
    # Ensure arrays
    y_train_pred  = np.asarray(y_train_pred,  dtype=float)
    y_train_pred1 = np.asarray(y_train_pred1, dtype=float).ravel()
    y_train_pred2 = np.asarray(y_train_pred2, dtype=float).ravel()
    pp            = np.asarray(pp,            dtype=float).ravel()
    cldvel        = np.asarray(cldvel,        dtype=float).ravel()

    # Sizes
    n1 = y_train_pred1.size
    if n1 % 13 != 0:
        raise ValueError("length(y_train_pred1) must be a multiple of 13")
    lm = n1 // 13
    V  = cldvel.size

    # Helper to place a linearized vector into rows 12:24 of a (24 x LM) matrix, MATLAB-style (Fortran order)
    def place_into_24xLM(vec):
        vec = np.asarray(vec, dtype=float).ravel()
        if vec.size != n1:
            raise ValueError("Input vector length mismatch.")
        M = np.full((24, lm), np.nan, dtype=float)
        M[11:24, :] = np.reshape(vec, (13, lm), order='F')  # rows 12..24 in MATLAB
        return M.T  # -> (LM, 24)

    # 1) PBL matrix (pp)
    pbltrain2 = place_into_24xLM(pp)

    # 2) BLC flags
    blc_f = place_into_24xLM(y_train_pred1)

    # 3) Cloud-base heights
    cldcbh1 = place_into_24xLM(y_train_pred2)

    # 4) Cloud frequency for 11 features (cols 0..10 of y_train_pred)
    cldcf = np.full((lm, 24, 11), np.nan, dtype=float)
    for k in range(11):
        vec = np.asarray(y_train_pred[:, k], dtype=float).ravel()
        Bk = np.full((24, lm), np.nan, dtype=float)
        Bk[11:24, :] = np.reshape(vec, (13, lm), order='F')
        cldcf[:, :, k] = Bk.T  # (LM, 24)

    # 5) Cloud thickness from feature 12 (col index 11)
    vec_th = np.asarray(y_train_pred[:, 11], dtype=float).ravel()
    T = np.full((24, lm), np.nan, dtype=float)
    T[11:24, :] = np.reshape(vec_th, (13, lm), order='F')
    cldth = T.T  # (LM, 24)

    # 6) Assemble final cloud train (LM x 24 x V)
    thresh = 0.5
    cldtrain = np.full((lm, 24, V), np.nan, dtype=float)

    for i in range(lm):
        for j in range(24):
            # If BLC flag < threshold => zeros; else construct profile
            if blc_f[i, j] < thresh:
                cldtrain[i, j, :] = 0.0
                continue

            cb = cldcbh1[i, j]
            th = cldth[i, j]

            # Guard against NaNs (MATLAB would error downstream; choose zeros here)
            if not np.isfinite(cb) or not np.isfinite(th):
                cldtrain[i, j, :] = 0.0
                continue

            if th >= 50:
                ct = cb + th - 1.0
                ix = int(np.argmin(np.abs(cldvel - cb)))  # nearest to base
                iy = int(np.argmin(np.abs(cldvel - ct)))  # nearest to top
                if iy < ix:
                    ix, iy = iy, ix
                ix = max(0, min(ix, V - 1))
                iy = max(0, min(iy, V - 1))

                c = np.squeeze(cldcf[i, j, :]).astype(float)  # length 11
                a = np.zeros(V, dtype=float)

                if iy >= ix:
                    # 11 points spanning [ix, iy], like ix : (iy-ix)/10 : iy in MATLAB
                    xi = np.linspace(ix, iy, 11)
                    grid = np.arange(ix, iy + 1)
                    # Linear interpolation of the 11 CF values across the index span
                    a[ix:iy + 1] = np.interp(grid, xi, c)
                cldtrain[i, j, :] = a
            else:
                # thin cloud: fill two levels with mean CF
                m = int(np.argmin(np.abs(cldvel - cb)))
                a = np.squeeze(cldcf[i, j, :]).astype(float)  # length 11
                mean_val = np.mean(a)  # match MATLAB mean (NaNs propagate)
                z = np.zeros(V, dtype=float)
                end_idx = min(m + 2, V)  # set indices m and m+1 if available
                z[m:end_idx] = mean_val
                cldtrain[i, j, :] = z

    return pbltrain2, blc_f, cldcbh1, cldcf, cldth, cldtrain

MODELDIR = './models'

with open(os.path.join(MODELDIR, 'pblh_candidate.pkl'), 'rb') as f:
    data = pickle.load(f)
    y_scaler = data['y_scaler']
    x_scaler = data['x_scaler']
    importances = data['importances']
    
features_to_keep = [i for i, importance in enumerate(importances) if importance < 0]
x_scaler_pblh=x_scaler

print(len(features_to_keep))

# Use only the models that were actually selected/used
selected_model_filenames = [
    'possible_candidate1.h5',
    'model_ff_1.h5',
    'model_ff_2.h5',
    'model_fff_0.h5',
    'model_fff_1.h5',
    'model_fff_3.h5',
    'model_fff_5.h5',
    'model_fff_8.h5',
    'model_fff_15.h5',
    'model_fff_17.h5',
]

# Load the data
data = sio.loadmat('input_for_dnn.mat', variable_names=['xx','xx1'])
xx = data['xx']
x_train1 = xx.T
        
# Scale the data
if rescale == 1:
    x_scaler_pblh = StandardScaler().fit(x_train1)  # <-- only when rescale==1
x_trainn = x_scaler_pblh.transform(x_train1)
    
# Define the batch size
batch_size = 1000

# Initialize an array to store predictions from selected models
y_pred_originalall = np.zeros((len(selected_model_filenames), x_trainn.shape[0], 1))

# Iterate over each selected model
for idx, model_filename in enumerate(selected_model_filenames):
    # Load the model
    model = load_model(os.path.join(MODELDIR, model_filename))

    y_pred_list = []
    for i in range(0, x_trainn.shape[0], batch_size):
        batch = x_trainn[i:i+batch_size, features_to_keep]  # Adjust features_to_keep if needed
        batch_pred = model.predict(batch)
        y_pred_list.append(batch_pred)

    y_pred_combined = np.vstack(y_pred_list)
    y_pred_original = y_scaler.inverse_transform(y_pred_combined)  # Assuming y_scaler is already fitted
    y_pred_originalall[idx, :] = y_pred_original
    
    y_pred_originalall1 = y_pred_originalall
    
    
    xx1 = data['xx1']
pp1 = np.squeeze(y_pred_originalall1)  # Remove single-dimensional entries
pp2 = np.mean(pp1, axis=0)  # Calculate the mean along the default axis
pp3 = np.insert(pp2[:-1], 0, np.nan)
oo = np.column_stack((pp3, pp2))
oo1=oo.T
oo2 = np.vstack((xx1, oo1))  # Vertically stack xx and xx1, then transpose
oo3 = oo1 - xx1[12:14,:]  # Difference between column pairs
xx2 = np.vstack((oo2, oo3))  # Vertically stack xx and xx1, then transpose
#xx = xx2[:, 1:]
xx=xx2
# Calculate the mean of all variables (columns)
means = np.mean(xx, axis=0)
print(means)
print(xx.shape)

model1 = load_model(os.path.join(MODELDIR, 'type_model_f1.h5'))
modelcf = load_model(os.path.join(MODELDIR, 'cfra_all.h5'))
modelcbh = load_model(os.path.join(MODELDIR, 'cbh_ff.h5'))

with open(os.path.join(MODELDIR, 'cfra_all.pkl'), 'rb') as f:
    data = pickle.load(f)
    important_features_indices = data['important_features_indices']
    features_to_keep = data['features_to_keep']
    x_scaler_cf = data['x_scaler_cf']
    y_scaler_cf = data['y_scaler_cf']
    x_scaler_type = data['x_scaler_type']

with open(os.path.join(MODELDIR, 'cbhf_all.pkl'), 'rb') as f:
    data = pickle.load(f)
    features_cbh = data['features_cbh']
    x_scaler_cbh = data['x_scaler_cbh']
    y_scaler_cbh = data['y_scaler_cbh']
    
x_train1 = xx.T

if rescale == 1:
    x_scaler_type = StandardScaler().fit(x_train1)
x_trainn = x_scaler_type.transform(x_train1)
y_train_pred1 = model1.predict(x_trainn[:, important_features_indices])
y_train_pred1=y_train_pred1.T
combined_matrix = np.vstack((xx, y_train_pred1))
x_train1 = combined_matrix.T

if rescale == 1:
    x_scaler_cbh = StandardScaler().fit(x_train1)
x_trainn = x_scaler_cbh.transform(x_train1)
y_train_pred = modelcbh.predict(x_trainn[:,features_cbh])
y_train_pred2 = y_scaler_cbh.inverse_transform(y_train_pred)

if rescale == 1:
    x_scaler_cf = StandardScaler().fit(x_train1)
x_trainn = x_scaler_cf.transform(x_train1)
y_train_pred = modelcf.predict(x_trainn[:,features_to_keep])
y_train_pred = y_scaler_cf.inverse_transform(y_train_pred)

# ---- prep shapes without any .mat I/O ----
y_train_pred  = np.asarray(y_train_pred)
y_train_pred1 = np.asarray(y_train_pred1).ravel()
y_train_pred2 = np.asarray(y_train_pred2).ravel()
pp            = np.asarray(pp2).ravel()

# Normalize shapes/orientation just in case
n1 = np.size(y_train_pred1)
y_train_pred1 = np.asarray(y_train_pred1).ravel()
y_train_pred2 = np.asarray(y_train_pred2).ravel()
pp            = np.asarray(pp).ravel()

# If y_train_pred is transposed (F x N), fix it to (N x F)
if y_train_pred.shape[0] != n1 and y_train_pred.shape[1] == n1:
    y_train_pred = y_train_pred.T

assert y_train_pred.shape[0] == n1, f"y_train_pred first dim {y_train_pred.shape[0]} != {n1}"
assert y_train_pred.shape[1] >= 12, "y_train_pred must have at least 12 columns (11 CF + thickness)."

# --- 2) Define cldvel like MATLAB: 30:30:150*30 ---
cldvel = np.arange(30, 150*30 + 1, 30, dtype=float)  # 30, 60, ..., 4500

# --- 3) Run build_cldtrain ---
pbltrain2, blc_f, cldcbh1, cldcf, cldth, cldtrain = build_cldtrain(
    y_train_pred, y_train_pred1, y_train_pred2, pp, cldvel
)


# --- Trim helpers (keep last 14 hours: 11..24) ---
def trim14(x):
    if x.ndim == 2:   # (LM, 24)
        return x[:, -14:]
    elif x.ndim == 3: # (LM, 24, K)
        return x[:, -14:, :]
    else:
        raise ValueError(f"Unexpected ndim={x.ndim} for trim14")

# Apply trim to all hour-sized arrays
pblh_14   = trim14(pbltrain2)     # (LM,14)
blc_14    = trim14(blc_f)         # (LM,14)
cbh_14    = trim14(cldcbh1)       # (LM,14)
cldcf_14  = trim14(cldcf)         # (LM,14,11)
cldth_14  = trim14(cldth)         # (LM,14)
cldprof_14= trim14(cldtrain)      # (LM,14,150)

SURF_PATH = 'input_for_dnn.mat'
surf = sio.loadmat(SURF_PATH, squeeze_me=True)
date_vec = np.ravel(surf['date']).astype(np.int64)

# --- Save outputs to ./output ---
outdir = './output'
os.makedirs(outdir, exist_ok=True)

# MAT
mat_path = os.path.join(outdir, 'dnn_output.mat')
payload = {
    'pblh'         : pblh_14,
    'blc_occurrence': blc_14,
    'cbh'          : cbh_14,
    'cldcf'        : cldcf_14,
    'cldthick'     : cldth_14,
    'cld_prof'     : cldprof_14,
    'height'       : cldvel,      # unchanged vertical grid
    'date' : date_vec,
}
sio.savemat(mat_path, payload)
print(f"Saved: {mat_path}")

# NetCDF
nc_path = os.path.join(outdir, 'dnn_output.nc')
LM, H14 = pblh_14.shape
V = cldprof_14.shape[2]
CFBINS = cldcf_14.shape[2]

with Dataset(nc_path, 'w', format='NETCDF4') as nc:
    # Dimensions
    nc.createDimension('time', LM)
    nc.createDimension('hour', H14)       # 14 hours
    nc.createDimension('height', V)       # 150
    nc.createDimension('cfbin', CFBINS)   # 11

    # Coordinates
    v_time   = nc.createVariable('time', 'i4', ('time',))
    v_height = nc.createVariable('height', 'f4', ('height',))
    v_time[:] = np.arange(LM, dtype=np.int32)
    v_height[:] = cldvel.astype(np.float32)

    if date_vec is not None:
        v_date = nc.createVariable('date_yyyymmdd', 'i8', ('time',))
        v_date[:] = date_vec.astype(np.int64)

    def makevar(name, dtype, dims):
        return nc.createVariable(name, dtype, dims, zlib=True, complevel=4, shuffle=True)

    makevar('pblh',          'f4', ('time','hour'))[:]         = pblh_14.astype(np.float32)
    makevar('blc_occurrence','f4', ('time','hour'))[:]         = blc_14.astype(np.float32)
    makevar('cbh',           'f4', ('time','hour'))[:]         = cbh_14.astype(np.float32)
    makevar('cldcf',         'f4', ('time','hour','cfbin'))[:] = cldcf_14.astype(np.float32)
    makevar('cldthick',      'f4', ('time','hour'))[:]         = cldth_14.astype(np.float32)
    makevar('cld_prof',      'f4', ('time','hour','height'))[:]= cldprof_14.astype(np.float32)

    # attrs
    nc.variables['height'].units = 'm'
    nc.variables['pblh'].units   = 'm'
    nc.variables['cbh'].units    = 'm'

print(f"Saved: {nc_path}")


58
24/24 [==============================] - 0s 728us/step
[        nan 81.75000275 81.89548069 ...         nan         nan
         nan]
(210, 4758)
  1/149 [..............................] - ETA: 15s

/global/common/software/nersc/pe/conda-envs/24.1.0/python-3.11/nersc-python/lib/python3.11/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


149/149 [==============================] - 0s 986us/step
Saved: ./output/dnn_output.mat
Saved: ./output/dnn_output.nc
